# 🎓 Session 1 - Introduction à l'IA et au Machine Learning
## Solutions avec Cas Pratique : Prédiction BAC Mauritanie 2022

**Formation IA & ML - SupNum Nouakchott**  
**Formateur:** Mohamed Beydia - Vela Learning  
**Dataset:** BAC Mauritanie 2022 Predictive Modeling Challenge

---

## 📋 Objectifs de la session

À la fin de cette session, vous serez capables de :
- [x] Définir l'Intelligence Artificielle, le Machine Learning et le Deep Learning
- [x] Distinguer l'IA symbolique de l'IA statistique
- [x] Comprendre la différence entre IA analytique et générative
- [x] Identifier les trois familles principales du ML
- [x] Décrire le workflow standard d'un projet ML
- [x] Reconnaître des cas d'usage concrets d'IA
- [x] **BONUS:** Analyser un dataset réel et créer votre première prédiction ML !

---

## 💻 7. PROJET PRATIQUE : Analyse BAC Mauritanie 2022

### 🎯 Notre Défi Kaggle
**Objectif :** Prédire les résultats du BAC 2022 en Mauritanie à partir des données des candidats

**Type de problème :** Classification supervisée  
**Métrique :** Accuracy, Precision, Recall

Commençons par explorer nos données !

In [ ]:
# Import des librairies essentielles pour notre projet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("✅ Librairies importées avec succès !")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

In [ ]:
# Extraction et chargement des données BAC Mauritanie
# Chemin vers le fichier zip (dans le dossier data/ de cette session)
zip_path = "data/bac-mauritanie-2022-predictive-modeling-challeng.zip"
extract_path = "data/extracted/"

# Créer le dossier d'extraction s'il n'existe pas
os.makedirs(extract_path, exist_ok=True)

# Extraire les données
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Données extraites avec succès !")
    
    # Lister les fichiers extraits
    files = os.listdir(extract_path)
    print("📁 Fichiers disponibles:")
    for file in files:
        print(f"  - {file}")
        
except Exception as e:
    print(f"❌ Erreur lors de l'extraction: {e}")
    print("Vérifiez que le fichier zip existe dans data/")

In [ ]:
# 🔧 CHARGEMENT ROBUSTE DES DONNÉES CSV
def load_csv_robust(file_path, encoding_attempts=['utf-8', 'latin-1', 'cp1252']):
    """
    Charge un fichier CSV de manière robuste avec gestion des erreurs
    """
    filename = file_path.split('/')[-1]
    print(f"🔄 Tentative de chargement: {filename}")
    
    for encoding in encoding_attempts:
        try:
            # Utiliser des paramètres plus permissifs pour gérer les erreurs
            df = pd.read_csv(file_path, 
                           encoding=encoding,
                           sep=',',
                           quotechar='"',
                           on_bad_lines='skip',  # Ignorer les lignes problématiques
                           engine='python',      # Moteur Python plus robuste
                           dtype=str)            # Tout charger en string d'abord
            print(f"✅ {filename} chargé - Encoding: {encoding} - {len(df)} lignes")
            return df
        except Exception as e:
            print(f"❌ Échec {encoding}: {str(e)[:100]}...")
            continue
    
    print(f"❌ Impossible de charger {filename}")
    return None

# Chargement des datasets avec gestion d'erreurs robuste
try:
    print("🔄 Chargement robuste des datasets...")
    print()
    
    # Essayer les différents noms possibles des fichiers
    train_files = ['train_set.csv', 'train.csv']
    test_files = ['test_set.csv', 'test.csv'] 
    submission_files = ['submission_template.csv', 'sample_template.csv']
    
    # Chargement du fichier d'entraînement
    train_df = None
    for filename in train_files:
        filepath = f"{extract_path}/{filename}"
        if os.path.exists(filepath):
            train_df = load_csv_robust(filepath)
            if train_df is not None:
                break
    
    # Chargement du fichier de test
    test_df = None
    for filename in test_files:
        filepath = f"{extract_path}/{filename}"
        if os.path.exists(filepath):
            test_df = load_csv_robust(filepath)
            if test_df is not None:
                break
    
    # Chargement du template de soumission
    sample_submission = None
    for filename in submission_files:
        filepath = f"{extract_path}/{filename}"
        if os.path.exists(filepath):
            sample_submission = load_csv_robust(filepath)
            if sample_submission is not None:
                break
    
    if train_df is not None and test_df is not None:
        print("\n✅ Datasets chargés avec succès !")
        print(f"📊 Données d'entraînement: {train_df.shape[0]} lignes, {train_df.shape[1]} colonnes")
        print(f"📊 Données de test: {test_df.shape[0]} lignes, {test_df.shape[1]} colonnes")
        if sample_submission is not None:
            print(f"📊 Template de soumission: {sample_submission.shape}")
        
        # Convertir les colonnes numériques
        print("\n🔄 Conversion des types de données...")
        
        # Identifier et convertir les colonnes de notes
        numeric_columns = []
        for col in train_df.columns:
            if any(keyword in col.lower() for keyword in ['note', 'moyenne', 'general']):
                try:
                    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
                    if col in test_df.columns:
                        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')
                    numeric_columns.append(col)
                    print(f"  ✓ {col} converti en numérique")
                except:
                    print(f"  ❌ Impossible de convertir {col}")
        
        print(f"\n📊 Colonnes numériques identifiées: {len(numeric_columns)}")
        
        # Vérifier la qualité des données
        print("\n🔍 Vérification de la qualité:")
        print(f"  - Valeurs manquantes train: {train_df.isnull().sum().sum()}")
        print(f"  - Valeurs manquantes test: {test_df.isnull().sum().sum()}")
        
    else:
        print("❌ Problème de chargement des datasets")
        print("🔍 Fichiers disponibles dans le dossier extracted:")
        for file in os.listdir(extract_path):
            file_size = os.path.getsize(f"{extract_path}/{file}") / 1024  # KB
            print(f"  - {file} ({file_size:.1f} KB)")
        
except Exception as e:
    print(f"❌ Erreur générale: {e}")
    print("\n💡 Solutions:")
    print("1. Vérifiez que le fichier zip est bien extrait")
    print("2. Regardez les noms exacts des fichiers CSV")
    print("3. Le dataset peut contenir des caractères spéciaux")

In [ ]:
# 🔍 EXPLORATION INITIALE DES DONNÉES
if train_df is not None:
    print("🔍 === EXPLORATION DES DONNÉES BAC MAURITANIE 2022 ===")
    print()
    
    # Aperçu des données
    print("📋 Aperçu des premières lignes:")
    print(train_df.head())
    print()
    
    # Informations sur les colonnes
    print("📊 Informations sur les colonnes:")
    print(f"Nombre total de colonnes: {len(train_df.columns)}")
    print("Noms des colonnes:")
    for i, col in enumerate(train_df.columns):
        dtype = train_df[col].dtype
        non_null = train_df[col].count()
        print(f"  {i+1:2d}. {col:25s} | {str(dtype):10s} | {non_null:5d} non-null")
    
    # Identifier la variable cible
    target_columns = [col for col in train_df.columns if any(word in col.lower() for word in ['decision', 'result', 'target', 'label'])]
    if target_columns:
        target_col = target_columns[0]
        print(f"\n🎯 Variable cible identifiée: '{target_col}'")
        
        # Distribution de la variable cible
        print("\n📊 Distribution de la variable cible:")
        value_counts = train_df[target_col].value_counts()
        for value, count in value_counts.items():
            percentage = (count / len(train_df)) * 100
            print(f"  {value}: {count} ({percentage:.1f}%)")
    else:
        print("\n⚠️ Variable cible non identifiée automatiquement")
        print("Colonnes possibles:", list(train_df.columns)[:10], "...")
        
else:
    print("❌ Impossible d'explorer les données - chargement échoué")

In [ ]:
# 📊 ANALYSE VISUELLE SIMPLE
if train_df is not None and target_columns:
    try:
        target_col = target_columns[0]
        
        # Distribution de la variable cible
        plt.figure(figsize=(12, 5))
        
        # Graphique en barres
        plt.subplot(1, 2, 1)
        value_counts = train_df[target_col].value_counts()
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
        value_counts.plot(kind='bar', color=colors[:len(value_counts)])
        plt.title('Distribution des Résultats BAC 2022', fontweight='bold')
        plt.xlabel('Décision')
        plt.ylabel('Nombre d\'étudiants')
        plt.xticks(rotation=45)
        
        # Graphique circulaire
        plt.subplot(1, 2, 2)
        plt.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%',
                colors=colors[:len(value_counts)])
        plt.title('Répartition des Résultats', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Analyse des notes si disponibles
        note_columns = [col for col in train_df.columns if 'note' in col.lower()]
        if note_columns:
            print(f"\n📝 Colonnes de notes identifiées: {len(note_columns)}")
            print(note_columns)
            
            # Statistiques des notes
            if len(note_columns) > 0:
                # Sélectionner quelques colonnes de notes pour l'analyse
                sample_notes = note_columns[:min(6, len(note_columns))]
                
                # Créer un DataFrame numérique pour les notes
                notes_df = train_df[sample_notes].apply(pd.to_numeric, errors='coerce')
                
                print("\n📊 Statistiques des notes:")
                print(notes_df.describe().round(2))
                
                # Visualisation des notes
                if len(sample_notes) >= 4:
                    plt.figure(figsize=(15, 8))
                    for i, col in enumerate(sample_notes[:4], 1):
                        plt.subplot(2, 2, i)
                        notes_df[col].hist(bins=20, alpha=0.7, color=f'C{i-1}')
                        plt.title(f'Distribution {col}')
                        plt.xlabel('Note')
                        plt.ylabel('Fréquence')
                        mean_val = notes_df[col].mean()
                        if not pd.isna(mean_val):
                            plt.axvline(mean_val, color='red', linestyle='--', 
                                      label=f'Moyenne: {mean_val:.1f}')
                            plt.legend()
                    
                    plt.suptitle('Distribution des Notes - BAC Mauritanie 2022', fontsize=14, fontweight='bold')
                    plt.tight_layout()
                    plt.show()
        
        print("\n✅ Analyse visuelle terminée !")
        
    except Exception as e:
        print(f"❌ Erreur lors de la visualisation: {e}")
        print("Continuons avec l'analyse textuelle...")
        
else:
    print("⚠️ Pas de données à visualiser")

In [ ]:
# 🎯 RÉCAPITULATIF DE L'EXPLORATION
if train_df is not None:
    print("🎯 === RÉCAPITULATIF DE L'ANALYSE ===")
    print()
    
    print(f"✅ Dataset BAC Mauritanie 2022 chargé avec succès")
    print(f"📊 {len(train_df)} étudiants dans le dataset d'entraînement")
    print(f"📊 {len(train_df.columns)} caractéristiques par étudiant")
    
    if target_columns:
        target_col = target_columns[0]
        unique_decisions = train_df[target_col].nunique()
        print(f"🎯 {unique_decisions} types de décisions possibles")
        
        # Calculer le taux de réussite si possible
        if any(word in train_df[target_col].str.lower().str.cat(sep=' ') for word in ['admis', 'reussi', 'pass']):
            success_rate = train_df[target_col].str.contains('admis|reussi|pass', case=False).mean()
            print(f"🏆 Taux de réussite estimé: {success_rate:.1%}")
    
    # Qualité des données
    missing_percentage = (train_df.isnull().sum().sum() / (len(train_df) * len(train_df.columns))) * 100
    print(f"📈 Qualité des données: {100-missing_percentage:.1f}% (complétude)")
    
    note_columns = [col for col in train_df.columns if 'note' in col.lower()]
    if note_columns:
        print(f"📚 {len(note_columns)} matières identifiées")
    
    print(f"\n💡 PROCHAINES ÉTAPES:")
    print(f"   1. Nettoyage et préparation des données")
    print(f"   2. Feature engineering (création de nouvelles variables)")
    print(f"   3. Entraînement de modèles ML")
    print(f"   4. Évaluation et optimisation")
    print(f"   5. Génération de prédictions pour Kaggle")
    
    print(f"\n🎉 Session 1 terminée avec succès !")
    print(f"📚 Prochaine session: Régression et prédiction des notes individuelles")
    
else:
    print("❌ Impossible de générer le récapitulatif - problème de chargement des données")
    print("\n🛠️ Actions recommandées:")
    print("   1. Vérifier que le fichier ZIP est présent dans data/")
    print("   2. Vérifier l'intégrité du fichier ZIP")
    print("   3. Examiner manuellement les fichiers CSV extraits")
    print("   4. Contacter le formateur si le problème persiste")

## 🎯 8. Récapitulatif et Réflexions

### ✅ Ce que nous avons accompli aujourd'hui :

- [x] **Théorie ML** : Définitions, types d'IA, familles ML, workflow
- [x] **Chargement robuste** : Gestion des erreurs CSV avec multiple encodings
- [x] **Exploration des données** : Structure, qualité, distribution
- [x] **Visualisations** : Graphiques pour comprendre les données
- [x] **Analyse BAC Mauritanie** : Dataset éducatif réel
- [x] **Préparation** : Base solide pour les sessions suivantes

### 🔧 Problèmes résolus :

1. **Erreur CSV Tokenization** : Résolu avec `on_bad_lines='skip'` et `engine='python'`
2. **Encodage des caractères** : Gestion multiple encodings (utf-8, latin-1, cp1252)
3. **Noms de fichiers variables** : Recherche automatique des bons noms
4. **Types de données** : Conversion robuste des colonnes numériques
5. **Gestion d'erreurs** : Code résistant aux problèmes de données

### 💡 Leçons apprises :

- **Les données réelles sont imparfaites** : Il faut toujours prévoir la gestion d'erreurs
- **L'encodage est crucial** : Les caractères spéciaux causent souvent des problèmes
- **La robustesse prime** : Un code qui gère les exceptions est essentiel
- **L'exploration avant tout** : Comprendre ses données avant de modéliser

### 🚀 Prochaines étapes :

**Session 2 - Régression et Prédiction**
- Nettoyer et préparer les données BAC
- Créer un modèle de régression pour prédire les notes
- Appliquer Ridge et Lasso pour la régularisation
- Évaluer avec RMSE, MAE, R²

---

**📧 Contact :** mohamed.beydia@vela-learning.com  
**🌐 Vela Learning :** [https://vela-learning.com](https://vela-learning.com)  
**🎓 SupNum Nouakchott**

**🎉 Session 1 terminée avec succès ! Données chargées et explorées ! 🎉**